In [5]:
import numpy as np

# Existing array
existing_array = np.empty([1, 3])
# New row to append
new_row = np.array([7, 8, 9])

# Append the new row using append
new_array = np.append(existing_array, [new_row], axis=0)

# Display the result
print("Existing Array:")
print(existing_array)

print("\nNew Row:")
print(new_row)

print("\nNew Array after appending the row:")
print(new_array[1:])


Existing Array:
[[7. 8. 9.]]

New Row:
[7 8 9]

New Array after appending the row:
[[7. 8. 9.]]


In [ ]:
import matplotlib.pyplot as plt
import cv2

from skimage import data
from skimage.filters import try_all_threshold

img = cv2.imread('tornillo.jpg')
# Convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

fig, ax = try_all_threshold(gray, figsize=(10, 8), verbose=False)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np

from skimage.filters import threshold_yen, threshold_mean, try_all_threshold, threshold_triangle, threshold_local
from skimage.morphology import skeletonize
from skimage.measure import label, regionprops, regionprops_table

img = cv2.imread('tornillo.jpg')
# Convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# Apply thresholding (adjust the parameters based on your image)

yen = threshold_triangle(gray)
#image = np.bitwise_not(gray > yen)
image = gray > yen
image = image.astype(np.uint8) * 255

kernel_size = 1
kernel = np.ones((kernel_size, kernel_size), np.uint8)
#image = cv2.dilate(cv2.erode(image, kernel, iterations=1), kernel, iterations=1)
image = cv2.erode(cv2.dilate(image, kernel, iterations=1), kernel, iterations=1)

label_image = label(image)
regions = regionprops(label_image)

plt.figure(figsize=(20, 20))

plt.subplot(2,2,1), plt.imshow(gray, cmap=plt.cm.gray)

ax = plt.subplot(2,2,2) 
ax.imshow(image, cmap=plt.cm.gray)

main_label = 0
area = 0
perimeter = 0
eccentricity = 0
moments_hu = 0
centroid = None
for props in regions:
    if props.area > area:
        main_label = props.label
        area = props.area
        perimeter = props.perimeter
        eccentricity = props.eccentricity 
        moments_hu = props.moments_hu
        centroid = props.centroid

y0, x0 = centroid
ax.plot(x0, y0, '.g', markersize=30)

print("Area: ", area)
print("Perimeter: ", perimeter)
print("Eccentricity: ", eccentricity)
print("Hu moments: ", moments_hu)
print("Centroid: ", centroid)
print(*moments_hu)
n = np.array([])
print(np.array([*moments_hu]))
n = np.concatenate((n, np.array([*moments_hu])), axis=0)

plt.subplot(2,2,3), plt.imshow(np.where(label_image == main_label, 1, 0), cmap=plt.cm.gray)

plt.show()

In [ ]:
import cv2
import numpy as np
from skimage.morphology import skeletonize
from matplotlib import pyplot as plt
from skimage.measure import label, regionprops, regionprops_table

# Read the image
image = cv2.imread('clavo.jpg')

# Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply thresholding (adjust the parameters based on your image)
_, binary_image = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY)
binary_image = cv2.bitwise_not(binary_image)

kernel_size = 10
kernel = np.ones((kernel_size, kernel_size), np.uint8)
open_img = cv2.erode(cv2.dilate(binary_image, kernel, iterations=1), kernel, iterations=1)
close_img = cv2.dilate(cv2.erode(open_img, kernel, iterations=1), kernel, iterations=1)

skeleton = skeletonize(binary_image)

label_img = label(binary_image)
regions = regionprops(label_img)
print(regions)
# display results
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(8, 4),
                         sharex=True, sharey=True)

ax = axes.ravel()

ax[0].imshow(binary_image, cmap=plt.cm.gray)
ax[0].axis('off')
ax[0].set_title('original', fontsize=20)

ax[1].imshow(skeleton, cmap=plt.cm.gray)
ax[1].axis('off')
ax[1].set_title('skeleton', fontsize=20)

fig.tight_layout()
plt.show()
